![logos](https://user-images.githubusercontent.com/14944147/120395211-79b4e980-c2e9-11eb-80af-7012b2e75cde.png)

# Practical Demonstration of New User-Requested Nav2 Features #


## Who Am I?

Hi, I'm [Steve Macenski](https://www.linkedin.com/in/steve-macenski-41a985101/). I do a great deal of different "robotics stuff" at [Samsung Research America](https://www.sra.samsung.com/) as the *Open-Source Robotics, Engineering Lead*. I consult with internal Samsung robotics teams on technology and business, work on mobile robot and perception research, and lead the ROS2 mobile robotics ecosystem. I sit on the [ROS2 TSC](https://docs.ros.org/en/foxy/Governance.html) and I am the leader of the Navigation Working Group. 

Before my current role, I was the leader of the robotics team at [Simbe Robotics](https://www.simberobotics.com/) and worked on NASA's Asteroid Redirect Robotic Mission and RESTORE-L (a  servicing mission to Landsat-7).

## Overview

Nav2 is the second generation of the ROS Navigation Stack. It contains a growing set of capabilities, algorithms, and features to build production and research applications. You can find more information about Nav2 at: https://navigation.ros.org.

In the following talk, we're going to discuss some of the more basic new features in Nav2 that can help you build your research or applications. This includes:
    - Basic Nav2 operation through Rviz2 using **NavigateToPose**
    - Basic Nav2 operation through a Python3 script using **NavigateThroughPoses**
    - Use of the Waypoint Follower and Task Executor plugins
    - Introduction to keepout zones and speed restricted zones

We will then put all of these together to show a basic autonomous robotics demo based on Nav2. We will be doing this in a simulated warehouse where robots are often deployed:

![](https://raw.githubusercontent.com/aws-robotics/aws-robomaker-small-warehouse-world/master/docs/images/small_warehouse_gazebo.png)

*Taken from [here](https://github.com/aws-robotics/aws-robomaker-small-warehouse-world).

## Nav2 Overview

- Independent, modular servers
- Configurable navigation logic, dynamically loaded BT nodes, and per-task behavior selection
- A growing set of run-time reconfigurable algorithms and plugins
- Software quality, linting, and testing (86% test coverage)
- Documentation and a rich community with Slack and Working Group (Join us! [Sign up for the working group here](https://calendar.google.com/calendar/u/0/embed?src=agf3kajirket8khktupm9go748@group.calendar.google.com&ctz=America/Los_Angeles))


![](https://navigation.ros.org/_images/architectural_diagram.png)

## Let's Get Started!